In [2]:
import findspark
findspark.init()

In [3]:
from pyspark import SparkContext
sc = SparkContext("local", "BroadCast")

22/03/02 00:40:20 WARN Utils: Your hostname, ubuntu-virtual-machine resolves to a loopback address: 127.0.1.1; using 192.168.80.128 instead (on interface ens33)
22/03/02 00:40:20 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/03/02 00:40:21 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/03/02 00:40:22 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [7]:
sector_dict = {
        "MSFT": "TECH",
        "TSLA": "AUTO",
        "EMR": "INDUSTRIAL"
}

stocks = [
    ("EMR", 52.0),
    ("TSLA", 300.0),
    ("MSFT", 100.0)
]

In [8]:
# create broadcasted variabel using Spark Context
# this will ensure that sector_dict is kept in every executor 
# where task shall be running
# lazy evaluation, data shall be copied to executors when we run the job
broadCastSectorDict = sc.broadcast(sector_dict)

In [9]:
stocksRdd = sc.parallelize(stocks)

In [10]:
# Pyspark see this code, this has reference to broadCastSectorDict
# which is broardcast data, pyspark place the broadCastSectorDict in every executor
# 1 time instead of every job
# without broadCast, sector_dict shall be copied to executor for every task
# add sector code with stock at executor level when task running
def enrichStockWithSector(stock):
    return stock + (broadCastSectorDict.value[stock[0]] ,)

In [11]:
# code marshalling - Python copy the lamnda code to executor system/processor
# now enrichStockWithSector also shipped to executor on every task
enrichedRdd = stocksRdd.map (lambda stock: enrichStockWithSector(stock))


In [12]:
enrichedRdd.collect()

[('MSFT', 100.0, 'TECH'), ('EMR', 52.0, 'INDUSTRIAL'), ('TSLA', 300.0, 'AUTO')]

In [13]:
# is rdd empty or not

enrichedRdd.isEmpty() # False

False

In [14]:
eRdd = sc.parallelize([])
eRdd.isEmpty() # True

True